## Machine Learning 프로젝트 수행을 위한 코드 구조화

`(분류, 회귀 Task)`

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [1]:
pip install -U scikit-learn

In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [3]:
# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = False ## feature extraction (curse of dimensionality)

In [4]:
# 데이터를 불러옵니다.
train = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2022/test.csv')
print(train.shape, test.shape)

In [5]:
train.columns

In [6]:
tr_col = ['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure']

##EDA
1. 데이터 크기 확인
2. 결측치 체크
3. dtype이 object인 column 확인
4. target distribution --> target value에 대한 분포 확인!

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

#### 결측치 처리

In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

 # Product_Code, ID 제거
train = train.drop(columns=['id','product_code'])

for i in range(len(train)): #앞에 Meterial_ 제거
  train['attribute_0'][i] = train['attribute_0'][i].split('_')[1]
  train['attribute_1'][i] = train['attribute_1'][i].split('_')[1]

# Min Max scaler
from sklearn.preprocessing import MinMaxScaler

scaled = MinMaxScaler().fit_transform(train)
train_scaled = pd.DataFrame(data=scaled, columns=train.columns, index=train.index)

# KNN Imputer
train_filled = imputer.fit_transform(train_scaled)
train = pd.DataFrame(train_filled)
train

In [8]:
#TEST 데이터 전처리
test = test.drop(columns=['id','product_code'])

for i in range(len(test)): #앞에 Meterial_ 제거
  test['attribute_0'][i] = test['attribute_0'][i].split('_')[1]
  test['attribute_1'][i] = test['attribute_1'][i].split('_')[1]

scaled = MinMaxScaler().fit_transform(test)
test_scaled = pd.DataFrame(data=scaled, columns=test.columns, index=test.index)

#KNN imputer 
test_filled = imputer.fit_transform(test_scaled)
test = pd.DataFrame(test_filled)
test

# 3. Training

In [9]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
# train : test = 8 : 2
from sklearn.model_selection import train_test_split

X = train.drop(columns=[23])
y = train[23]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,
                                                  random_state=42)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

In [10]:
# 2. fitting
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor

reg = LGBMRegressor()
reg2 = RandomForestRegressor()

reg.fit(X_train, y_train)

print("LightGBM params", reg.get_params())
print("\n\nRandomForest params", reg2.get_params())

In [11]:
# 3. evaluation

train_pred = reg.predict(X_train)

print("roc_auc_score : %.4f" % roc_auc_score(y_train, train_pred))

In [12]:
# 4. hyper-parameter tuning (AutoML framework)
import optuna

# objective 함수는 학습시킬 데이터 & 모델을 기준으로 어떤 Hyper-parameter를 tuning할 지 세팅해주는 함수.
## 정의한 데이터와 모델 모두 학습 및 평가까지 구현이 되어있어야 합니다.


def objective(trial):

    param = {
        'objective': 'mape',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    reg = LGBMRegressor(**param)
    reg.fit(X_train, y_train)
    pred = reg.predict(X_val)
    
    mape = mean_absolute_percentage_error(y_val, pred)
    
    return mape


# LGBM과 RF에 대한 study 변수들과 objective 함수들을 사용하여 hyper-parameter tuning을 수행합니다.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [13]:
study.trials_dataframe()

In [14]:
optuna.visualization.plot_optimization_history(study)

In [15]:
optuna.visualization.plot_param_importances(study)

In [16]:
# finalize
final_lgbm = LGBMRegressor(**study.best_params)
final_lgbm.fit(X, y)

print("Best params for LGBMRegressor" , final_lgbm.get_params())

In [17]:
## finalize check
print("roc_auc_score : %.4f" % roc_auc_score(y, final_lgbm.predict(X)))

In [22]:
# predict
result = final_lgbm.predict(test)
print(result.shape)

In [23]:
result

In [24]:
submission = pd.read_csv('../input/tabular-playground-series-aug-2022/sample_submission.csv')
submission

In [25]:
submission["failure"] = result
submission["failure"].value_counts()

In [27]:
submission

In [29]:
version_x = input()
submission.to_csv(f"./submission_{version_x}.csv", index=False)